# ショアの素因数分解アルゴリズムを学習する

ここではショアのアルゴリズムを学習します。アルゴリズムの元になっている量子位相推定の手法を学んだ後、ショアのアルゴリズムの各ステップを実例とともに紹介します。最後に、Qiskitを使用してショアのアルゴリズムを実装し、実際に素因数分解を行ってみます。

## 目次

1. [はじめに](#introduction)
2. [量子位相推定](#qpe)
    1. [1量子ビットの位相推定](#qpe_1qubit)
    2. [$n$量子ビットの位相推定](#qpe_nqubit)
3. [ショアのアルゴリズム](#shor)
    1. [位相オラクルの導入](#grover_phaseoracle)




## はじめに <a id='introduction'></a>

古典計算の能力をはるかに上回る量子計算の一つの例として、最も有名なものがショアの量子計算アルゴリズムでしょう。このアルゴリズムは、大きな正の数を二つの素数の積に分解するというものです。古典計算では素因数分解の有効なアルゴリズムが知られておらず、数が大きくなると指数関数的に計算量が増えると予想されています。ショアのアルゴリズムを用いれば、同じ問題を多項式時間で解くことができると考えられています。
古典計算での素因数分解の難しさは鍵暗号技術の元になっており、指数関数的に高速なショアのアルゴリズムが大きく注目される理由もそこにあります。

## 量子位相推定 <a id='qpe'></a>

まず、ショアのアルゴリズムの元になっている「量子位相推定」（Quantum Phase Estimation, QPE）と呼ばれる手法について学びましょう（実際、ショアのアルゴリズムはほぼQPEそのものであることが追々分かります）。
QPEの理解には「量子フーリエ変換」（Quantum Fourier Transform, QFT）の理解が欠かせませんが、QFTについてはXXを参照してください。
QPEが考える問題は、あるユニタリー演算$U$に対して$U|\psi\rangle=e^{2\pi i\theta}|\psi\rangle$となる固有ベクトル$|\psi\rangle$が与えられるとして、その固有値$e^{2\pi i\theta}$の位相$\theta$を求めることができるか？という問題です。

### 1量子ビットの位相推定 <a id='qpe_1qubit'></a>
まず、下図にあるような量子回路を考えてみましょう。

<img src="figs/qpe_1qubit.png" width=50%>

この場合、量子回路のステップ 1-3での量子状態は以下のようになります。

- ステップ 1 : $\frac{1}{\sqrt{2}}(|0\rangle|\psi\rangle+|1\rangle|\psi\rangle)$
- ステップ 2 : $\frac{1}{\sqrt{2}}(|0\rangle|\psi\rangle+|1\rangle e^{2\pi i\theta}|\psi\rangle)$
- ステップ 3 : $\frac12\left[(1+e^{2\pi i\theta})|0\rangle+(1-e^{2\pi i\theta})|1\rangle\right]$

この状態で上側の量子ビットを測定すると、$|(1+e^{2\pi i\theta})/2|^2$の確率で０、$|(1-e^{2\pi i\theta})/2|^2$の確率で1を測定するでしょう。つまり、この確率の値から位相$\theta$を求めることができるわけです。
しかし、$\theta$の値が小さい場合、ほぼ100%の確率で0を、ほぼ0％の確率で1を測定することになるため、100%あるいは0%からのずれを精度良く決めるためには多数回の測定が必要なります。

### $n$量子ビットの位相推定 <a id='qpe_nqubit'></a>
そこで、上側のレジスタを$n$量子ビットに拡張した量子回路（下図）を考えてみましょう。

<img src="figs/qpe_wo_iqft.png" width=60%>

準備として、$U^{2^x}|\psi\rangle$が以下のように書けることに留意しておきます。

$$
\begin{aligned}
U^{2^x}|\psi\rangle&=U^{2^x-1}U|\psi\rangle\\
&=U^{2^x-1}e^{2\pi i\theta}|\psi\rangle\\
&=U^{2^x-2}e^{2\pi i\theta2}|\psi\rangle\\
&=\cdots\\
&=e^{2\pi i\theta2^x}|\psi\rangle
\end{aligned}
$$

この量子回路のステップ 1, 2, ... $n+1$での量子状態は以下のようになります。

- ステップ 1 : $\frac{1}{\sqrt{2^n}}(|0\rangle+|1\rangle)^{\otimes n}|\psi\rangle$
- ステップ 2 : $\frac{1}{\sqrt{2^n}}(|0\rangle+e^{2\pi i\theta2^{n-1}}|1\rangle)(|0\rangle+|1\rangle)^{\otimes n-1}|\psi\rangle$
- $\cdots$
- ステップ $n+1$ : $\frac{1}{\sqrt{2^n}}(|0\rangle+e^{2\pi i\theta2^{n-1}}|1\rangle)(|0\rangle+e^{2\pi i\theta2^{n-2}}|1\rangle)\cdots(|0\rangle+e^{2\pi i\theta2^0}|1\rangle)|\psi\rangle$

ステップ $n+1$後の$n$ビットレジスタの状態は、QFTで$j$を$2^n\theta$としたものと同等であることが分かります。つまり、この$n$ビットレジスタに逆フーリエ変換QFTを施せば、状態$|2^n\theta\rangle$が得られることになります。この状態を測定することで$2^n\theta$、つまり固有値の位相$\theta$（を$2^n$倍したもの）を求めることができる、というのがQPEです（下図）。

<img src="figs/qpe.png" width=80%>

## ショアのアルゴリズム <a id='shor'></a>

では、ショアのアルゴリズムの本題に入っていきましょう。ショアのアルゴリズムが考えるのは「ある正の合成数$N$を、自明ではない素数の積$N=qp$に分解する」という問題です。

まず、整数の剰余についての表記法をおさらいしておきます。以下のような自然数の並び$x$を考えたとき、例えば3で割った余りを$y$とすると

| x | 1 | 2 | 3 |
| y | 0 | 1 | 2 |

<img src="figs/shor_flow.png" width=60%>